In [1]:
import os
import re
import io
import sys
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import (
    accuracy_score, roc_auc_score, average_precision_score,
    confusion_matrix, classification_report, roc_curve, precision_recall_curve
)

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except Exception:
    HAS_XGB = False

from dotenv import load_dotenv, find_dotenv

In [2]:
# FUNÇÕES DE PREPARO

def coerce_numeric(s):
    return pd.to_numeric(s, errors="coerce")
def extrair_fase(valor):
    if pd.isna(valor):
        return np.nan
    valor = str(valor).lower()
    if "alfa" in valor:
        return 0
    m = re.search(r"fase\s*(\d+)", valor)
    if m:
        return int(m.group(1))
    return np.nan


def padronizar_genero(df):
    df = df.copy()
    if "genero" in df.columns:
        df["genero"] = df["genero"].astype(str).str.strip().str.lower()
        map_genero = {
            "menino": "masculino",
            "masculino": "masculino",
            "menina": "feminino",
            "feminino": "feminino"
        }
        df["genero"] = df["genero"].map(map_genero)
    return df


def padronizar_idade(df):
    df = df.copy()
    if "idade" not in df.columns:
        return df

    s = df["idade"]
    dt = pd.to_datetime(s, errors="coerce")

    idade_from_date = np.where(
        dt.notna() & (dt.dt.year == 1900) & (dt.dt.month == 1),
        dt.dt.day,
        np.nan
    )

    idade_num = pd.to_numeric(s, errors="coerce")
    idade_final = pd.Series(idade_num, index=df.index)

    mask = idade_final.isna() & ~pd.isna(idade_from_date)
    idade_final.loc[mask] = idade_from_date[mask]

    # faixa plausível
    idade_final = idade_final.where(idade_final.between(6, 30))
    df["idade"] = idade_final.round()
    return df


def tratar_inde_2024(df):
    df = df.copy()
    if "inde_2024" in df.columns:
        tmp = df["inde_2024"].astype(str).str.strip().str.upper()
        tmp = tmp.replace("INCLUIR", np.nan)
        df["inde_2024"] = coerce_numeric(tmp)
    return df


def preparar_base(df, modo_treino: bool):
    """
    modo_treino=True:
      - cria target a partir de 'ian'
      - remove vazamento (ian/defasagem)
    modo_treino=False:
      - não cria target
      - remove vazamento se existir
    """
    df = df.copy()

    df = padronizar_genero(df)
    df = padronizar_idade(df)
    df = tratar_inde_2024(df)

    if "fase_ideal" in df.columns:
        df["fase_ideal"] = df["fase_ideal"].apply(extrair_fase)

    # Target (somente treino)
    if modo_treino:
        if "ian" not in df.columns:
            raise ValueError("modo_treino=True exige coluna 'ian' para criar a target.")
        df["risco_defasagem_atual"] = (pd.to_numeric(df["ian"], errors="coerce") <= 5).astype(int)

    # Remover vazamento sempre que existir
    df = df.drop(columns=[c for c in ["ian", "defasagem"] if c in df.columns], errors="ignore")

    # Remoções iguais ao treino (mantém seu padrão)
    colunas_para_remover = ["ra",
                            "nome",
                            "data_nasc",
                            "escola",
                            "avaliador_1",
                            "avaliador_2",
                            "avaliador_3",
                            "avaliador_4",
                            "avaliador_5",
                            "avaliador_6",
                            "rec_av1",
                            "rec_av2",
                            "rec_av3",
                            "rec_av4",
                            "rec_av5",
                            "rec_av6",
                            "rec_psicologia",
                            "indicado",
                            "atingiu_pv",
                            "destaque_ieg",
                            "destaque_ida",
                            "destaque_ivp",
                            "pedra_2020",
                            "pedra_2021",
                            "pedra_2022",
                            "pedra_2023",
                            "pedra_2024",
                            "fase",
                            "turma",
                            "instituicao_ensino",
                            "ativo_inativo",
                            "cg",
                            "cf",
                            "ct",
                            "inde_2024"
    ]
    df = df.drop(columns=[c for c in colunas_para_remover if c in df.columns], errors="ignore")

    # Features extras
    cols_acad = [c for c in ["mat","por","ing"] if c in df.columns]
    if len(cols_acad) >= 2:
        df["media_academica"] = df[cols_acad].mean(axis=1)

    cols_comp = [c for c in ["iaa","ieg","ips","ipp"] if c in df.columns]
    if len(cols_comp) >= 2:
        df["media_comportamental"] = df[cols_comp].mean(axis=1)

    if ("inde_2022" in df.columns) and ("inde_2023" in df.columns):
        df["delta_inde"] = df["inde_2023"] - df["inde_2022"]

    return df


def make_preprocess(X_train: pd.DataFrame) -> ColumnTransformer:
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = [c for c in X_train.columns if c not in num_cols]

    numeric_pipe = Pipeline(steps=[
        ("imputer", KNNImputer(n_neighbors=7, weights="distance")),
        ("scaler", StandardScaler())
    ])

    categorical_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", categorical_pipe, cat_cols),
        ],
        remainder="drop"
    )
    return preprocess

In [3]:
# 2) TREINO COMPLETO + TESTE + THRESHOLD + SALVAR ARQUIVOS
# ============================================================

def treinar_e_salvar(excel_path: str,
                    out_model_path: str = "modelo_passos_magicos.pkl",
                    out_cfg_path: str = "config_passos_magicos.pkl",
                    seed: int = 42):

    base = pd.read_excel(excel_path)
    print("Shape original:", base.shape)

    base2 = preparar_base(base, modo_treino=True)
    print("Shape após preparo:", base2.shape)
    print("Target balance:\n", base2["risco_defasagem_atual"].value_counts())

    if "ano_pede" not in base2.columns:
        raise ValueError("Coluna 'ano_pede' não encontrada após preparo. Precisa dela para split temporal.")

    # Split temporal
    train_df = base2[base2["ano_pede"] < 2024].copy()
    test_df  = base2[base2["ano_pede"] == 2024].copy()

    X_train = train_df.drop(columns=["risco_defasagem_atual"])
    y_train = train_df["risco_defasagem_atual"].astype(int)

    X_test = test_df.drop(columns=["risco_defasagem_atual"])
    y_test = test_df["risco_defasagem_atual"].astype(int)

    print("Treino:", X_train.shape, "Teste:", X_test.shape)

    preprocess = make_preprocess(X_train)

    models = {
        "LogReg": LogisticRegression(max_iter=2000, class_weight="balanced"),
        "RandomForest": RandomForestClassifier(
            n_estimators=400, random_state=seed,
            class_weight="balanced_subsample"
        ),
        "MLP": MLPClassifier(
            hidden_layer_sizes=(64, 32),
            activation="relu",
            solver="adam",
            max_iter=500,
            random_state=seed
        ),
    }

    if HAS_XGB:
        models["XGBoost"] = XGBClassifier(
            n_estimators=600,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.9,
            colsample_bytree=0.9,
            reg_lambda=1.0,
            random_state=seed,
            eval_metric="logloss"
        )
    else:
        print("XGBoost não disponível. Seguindo sem XGBoost.")

    # CV no treino
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    scoring = {"acc": "accuracy", "roc_auc": "roc_auc", "pr_auc": "average_precision"}

    print("\n================= CV (TREINO) =================")
    cv_results = []
    pipes = {}

    for name, clf in models.items():
        pipe = Pipeline(steps=[("prep", preprocess), ("model", clf)])
        pipes[name] = pipe

        scores = cross_validate(
            pipe, X_train, y_train,
            cv=cv, scoring=scoring,
            n_jobs=-1, return_train_score=False
        )

        row = {
            "model": name,
            "acc_mean": scores["test_acc"].mean(),
            "acc_std": scores["test_acc"].std(),
            "roc_auc_mean": scores["test_roc_auc"].mean(),
            "roc_auc_std": scores["test_roc_auc"].std(),
            "pr_auc_mean": scores["test_pr_auc"].mean(),
            "pr_auc_std": scores["test_pr_auc"].std(),
        }
        cv_results.append(row)

        print(f"\n{name}")
        print(f"  ACC     : {row['acc_mean']:.4f} ± {row['acc_std']:.4f}")
        print(f"  ROC AUC : {row['roc_auc_mean']:.4f} ± {row['roc_auc_std']:.4f}")
        print(f"  PR AUC  : {row['pr_auc_mean']:.4f} ± {row['pr_auc_std']:.4f}")

    cv_df = pd.DataFrame(cv_results).sort_values("roc_auc_mean", ascending=False)
    print("\nResumo CV:")
    print(cv_df)

    # Teste final 2024
    print("\n================= TESTE FINAL (2024) =================")
    test_rows = []

    def avaliar(nome, pipe):
        pipe.fit(X_train, y_train)
        y_proba = pipe.predict_proba(X_test)[:, 1]
        y_pred = pipe.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        roc = roc_auc_score(y_test, y_proba)
        pr  = average_precision_score(y_test, y_proba)

        print(f"\n>>> {nome}")
        print(f"ACC     : {acc:.4f}")
        print(f"ROC AUC : {roc:.4f}")
        print(f"PR AUC  : {pr:.4f}")
        print("\nMatriz de confusão:")
        print(confusion_matrix(y_test, y_pred))
        print("\nClassification report:")
        print(classification_report(y_test, y_pred, digits=4))

        return {"model": nome, "acc": acc, "roc_auc": roc, "pr_auc": pr}

    for name, pipe in pipes.items():
        test_rows.append(avaliar(name, pipe))

    test_df_res = pd.DataFrame(test_rows).sort_values("roc_auc", ascending=False)
    print("\nResumo TESTE (ordenado por ROC AUC):")
    print(test_df_res)

    best_name = test_df_res.iloc[0]["model"]
    best_pipe = pipes[best_name]
    print("\nMelhor modelo no TESTE:", best_name)

    # Ajuste threshold no MELHOR modelo do teste
    best_pipe.fit(X_train, y_train)
    y_proba_best = best_pipe.predict_proba(X_test)[:, 1]

    thresholds = np.linspace(0.10, 0.90, 17)
    thr_rows = []
    for t in thresholds:
        y_pred_t = (y_proba_best >= t).astype(int)
        report = classification_report(y_test, y_pred_t, output_dict=True, zero_division=0)
        acc = accuracy_score(y_test, y_pred_t)
        prec1 = report["1"]["precision"]
        rec1 = report["1"]["recall"]
        f11 = report["1"]["f1-score"]
        thr_rows.append([t, acc, prec1, rec1, f11])

    thr_df = pd.DataFrame(thr_rows, columns=["threshold","accuracy","precision_risco","recall_risco","f1_risco"])
    print("\nTabela de thresholds (classe RISCO=1):")
    print(thr_df)

    best_thr_row = thr_df.sort_values("f1_risco", ascending=False).iloc[0]
    best_threshold = float(best_thr_row["threshold"])

    print("\nMelhor threshold (por F1 da classe 1):", best_threshold)
    print("Linha escolhida:\n", best_thr_row)

    # Salvar arquivos
    joblib.dump(best_pipe, out_model_path)
    joblib.dump({"threshold": best_threshold, "best_model": best_name}, out_cfg_path)

    print("\nArquivos salvos:")
    print(f"- {out_model_path}  (pipeline completo)")
    print(f"- {out_cfg_path}  (threshold + nome do modelo)")

    return best_name, best_threshold, cv_df, test_df_res, thr_df

In [ ]:
# Localiza e carrega as variáveis do arquivo .env automaticamente
load_dotenv(find_dotenv())

# Puxa o caminho da variável de ambiente
data_path = os.getenv("RAW_DATA_PATH")


MODO = "treino"  # "treino" ou "app"
EXCEL_PATH = data_path 
MODEL_PATH = "modelo_passos_magicos.pkl"
CFG_PATH = "config_passos_magicos.pkl"

if MODO == "treino":
    treinar_e_salvar(EXCEL_PATH, MODEL_PATH, CFG_PATH)

elif MODO == "app":
    # Streamlit NÃO é ideal no Colab, mas em ambiente local funciona:
    # streamlit run seu_arquivo.py
    print("Modo app: rode em ambiente local com Streamlit.")
    print("Exemplo: streamlit run projeto_completo.py")
else:
    raise ValueError("MODO inválido. Use 'treino' ou 'app'.")

Shape original: (860, 42)


In [ ]:
pipe = joblib.load('modelo_passos_magicos.pkl')
cfg = joblib.load('config_passos_magicos.pkl')

threshold = cfg['threshold']
print('Threshold: ', threshold)

base = pd.read_excel("/content/Base de Dados PEDE.xlsx")

base_prep = preparar_base(base, modo_treino=False)

sample = base_prep.sample(10, random_state=42)

proba = pipe.predict_proba(sample)[:, 1]
pred = (proba >= threshold).astype(int)

sample['proba'] = proba
sample['pred'] = pred

sample

In [ ]:
list(pipe.named_steps.keys())

In [ ]:
model = pipe.named_steps['model']

importances = model.feature_importances_

features = pipe.named_steps['prep'].get_feature_names_out()

imp = pd.DataFrame({
    'feature': features,
    'importance': importances
}).sort_values('importance', ascending=False)

imp.head(15)